# Data Isolation Sample Notebook

This notebook showcases the usage of data sources in MaximumGPT by using a local vector store with a single index and filter tags on different data sources. The solution is based on the schema described in the following picture and the setup of DS3. The sample consists of the following steps:
* Create the required MaximumGPT modules
* Define 2 assistants, which will be used to read different data from the same vector store index
* Define 2 data sources, each per assistant
* Raw text ingestion to have seperated sets of knowledge in the vector store index
* Showcasing chat operations on the data sources.

<div style="text-align:center;"><img alt="image" width="75%" src="img/concept_data_sources.png" /></div>

# Network & Imports

In [ ]:
import json
import requests
base_path= 'http://127.0.0.1:5002/api/v1'

# Use this if you are testing against an instance with OPENID auth
# %run fetch_openid_bearer.ipynb
# print (auth_headers)

def stream_response(r):
    r.encoding = 'utf-8'
    if r.status_code == 200:
        for line in r.iter_lines(): 
            json_data = json.loads(line.decode('utf-8'))
            if json_data['type'] == 'chunk':
                print(json_data['content'], end='')   
        
        print()   
    else: 
        print(r.text)
        
def stream_ingest(r):
  r.encoding = 'utf-8'
  if r.status_code == 200:
      for line in r.iter_lines(): 
          json_data = json.loads(line.decode('utf-8'))
          print (f"{json_data['currentStep']} of  {json_data['steps']}")
  else: 
      print(r.text)

# Setup MaximumGPT Modules

In [ ]:
# LLM Module
r = requests.post(f'{base_path}/module', json=
{
  "name": "GPT4o on Azure",
  "specId": "d7975f09-2b33-4ddf-bc29-cdc6c64a6781",
  "parameters": [
    {
      "name": "M_LLM_AZURE_OPENAI_API_KEY",
      "value": "*********"
    },{
      "name": "M_LLM_AZURE_OPENAI_API_VERSION",
      "value": "2023-07-01-preview"
    },{
      "name": "M_LLM_AZURE_OPENAI_DEPLOYMENT_NAME",
      "value": "aip4d-gpt4o"
    },{
      "name": "M_LLM_AZURE_OPENAI_ENDPOINT",
      "value": "https://aiphuahhaiopai04.openai.azure.com"
    },{
      "name": "M_LLM_AZURE_OPENAI_MODEL_NAME",
      "value": "gpt-4o"
    }
  ]
})

llm_id_gpt4o = r.json().get("id")
print("GPT4o ID: " + llm_id_gpt4o)

# EMBEDDING_MODEL Module
r = requests.post(f'{base_path}/module', json=
{
  "name": "E1",
  "specId": "846232c8-b6e9-4ee2-87ab-3e173768f06c",
  "parameters": [
    {
      "name": "M_EMB_AZURE_OPENAI_API_KEY",
      "value": "*********"
    },{
      "name": "M_EMB_AZURE_OPENAI_API_VERSION",
      "value": "2023-07-01-preview"
    },{
      "name": "M_EMB_AZURE_OPENAI_DEPLOYMENT_NAME",
      "value": "aiphuahhaiembb01"
    },{
      "name": "M_EMB_AZURE_OPENAI_ENDPOINT",
      "value": "https://aiphuahhaiopai01.openai.azure.com"
    },{
      "name": "M_EMB_AZURE_OPENAI_MODEL_NAME",
      "value": "text-embedding-ada-002"
    }
  ]
})

embedding_id = r.json().get("id")
print("Embedding model ID: " + embedding_id)

# VECTOR_STORE Module
r = requests.post(f'{base_path}/module', json=
{
  "name": "VS3",
  "specId": "2b929513-16fa-47a4-94b5-7036e16fa8dc",
  "parameters": [{
    "name": "M_VECTOR_STORE_LOCAL_INDEX_NAME",
    "value": "data_isolation_idx"
  }]
})

vector_store_index_id = r.json().get("id")
print("Vector Store Index ID: " + vector_store_index_id)

# FILE_STORAGE Module
r = requests.post(f'{base_path}/module', json=
{
  "name": "FS1",
  "specId": "b3a6b442-8123-49ee-b379-9ee1ff9099ae",
  "parameters": []
})

file_storage_id = r.json().get("id")
print("File Storage ID: " + file_storage_id)

# Setup Assistants

## System Instruction

In [ ]:
# First, we create a system instruction to push the model to restrict responses to the context knowledge only
r = requests.post(f'{base_path}/system-instruction', json={
  "name": "Enforce assistant knowledge",
  "message": """
    You must only respond using knowledge in the context of the user's request. Do try to answer the user prompt using other information than the provided context.
    """
})
system_instruction_id = r.json()['id']
print(f"System Instruction ID: {system_instruction_id}")

## Assistants
We are creating the assistants first without data source. They will be added via the assistants afterwards.

In [ ]:
r = requests.post(f'{base_path}/assistant', json={
  "name": "A1",
  "description": "An Assistant with its own knowledge",
  "systemInstructionId": system_instruction_id,
  "customSystemInstruction": "At the end of each response, always append this text: 'Proudly presented by knowledge of A1.'",
  "llmIds": [
    llm_id_gpt4o
  ],
  "dataSourceIds": [],
  "public": False
})
assistant_1_id = r.json()['id']
print(f"Assistant A1 ID: {assistant_1_id}")

r = requests.post(f'{base_path}/assistant', json={
  "name": "A2",
  "description": "An Assistant with its own knowledge",
  "systemInstructionId": system_instruction_id,
  "customSystemInstruction": "At the end of each response, always append this text: 'Proudly presented by knowledge of A2.'",
  "llmIds": [
    llm_id_gpt4o
  ],
  "dataSourceIds": [],
  "public": False
})
assistant_2_id = r.json()['id']
print(f"Assistant A2 ID: {assistant_2_id}")

## Data Sources
We are creating 2 data sources for the 2 different assistants now

In [ ]:
r = requests.post(f'{base_path}/assistant/{assistant_1_id}/data-source', json={
  "name": "DS3-A1",
  "embeddingModelId": embedding_id,
  "vectorStoreId": vector_store_index_id,
  "fileStorageId": file_storage_id
})
data_source_a1_id = r.json()['id']
print(f"Data Source ID (Tagged for A1): {data_source_a1_id}")

r = requests.post(f'{base_path}/assistant/{assistant_2_id}/data-source', json={
  "name": "DS3-A2",
  "embeddingModelId": embedding_id,
  "vectorStoreId": vector_store_index_id,
  "fileStorageId": file_storage_id
})
data_source_a2_id = r.json()['id']
print(f"Data Source ID (Tagged for A2): {data_source_a2_id}")

## Ingest test data
We feed each data source with a particular knowledge now.

In [ ]:
# Feed knowledge of A1
data={
    'llmId': llm_id_gpt4o,
    'dataSourceId': data_source_a1_id,
    'content': "The secret of Mr. DataMcDaFace is: A1",
    'overrides': []
}

r = requests.post(f'{base_path}/ingest/text', json=data)
stream_ingest(r)

# Feed knowledge of A2
data={
    'llmId': llm_id_gpt4o,
    'dataSourceId': data_source_a2_id,
    'content': "The secret of Mr. DataMcDaFace is: A2",
    'overrides': []
}

r = requests.post(f'{base_path}/ingest/text', json=data)
stream_ingest(r)


# Chat
Now, we are creating conversations for each assistant and ask it the same question, hoping that each provides the expected answer from the isolated data set they have access too.

## Create Conversations

In [ ]:
r = requests.post(f'{base_path}/assistant/{assistant_1_id}/conversation', json={})
conversation_1_id = r.json().get("id")
print("Conversation 1 ID: " + conversation_1_id)

r = requests.post(f'{base_path}/assistant/{assistant_2_id}/conversation', json={})
conversation_2_id = r.json().get("id")
print("Conversation 2 ID: " + conversation_2_id)

## Test chatting with our knowledge

In [ ]:
print("**** Setup *****")
assistant_1_json = requests.get(f'{base_path}/assistant/{assistant_1_id}').json()
assistant_1_ds_json = requests.get(f'{base_path}/data-source/{assistant_1_json['dataSourceIds'][0]}').json()
assistant_2_json = requests.get(f'{base_path}/assistant/{assistant_2_id}').json()
assistant_2_ds_json = requests.get(f'{base_path}/data-source/{assistant_2_json['dataSourceIds'][0]}').json()

c1_json = requests.get(f'{base_path}/conversation/{conversation_1_id}').json()
c2_json = requests.get(f'{base_path}/conversation/{conversation_2_id}').json()

print (f"Assistant '{assistant_1_json['name']}' --> Data source '{assistant_1_ds_json['name']}'")
print (f"\tConversation '{c1_json['name']}' --> Data source '{c1_json['dataSourceIds']}'")
print("")
print (f"Assistant '{assistant_2_json['name']}' --> Data source '{assistant_2_ds_json['name']}'")
print (f"\tConversation '{c2_json['name']}' --> Data source '{c2_json['dataSourceIds']}'")
print("****************")

In [ ]:
r = requests.post(f'{base_path}/conversation/{conversation_1_id}/message', json=
{
  "parts": [
    {
      "type": "TEXT",
      "content": "What is the secret of Mr. DataMcDaFace?"
    }
  ],
  "overrides": []
})
stream_response(r)

r = requests.post(f'{base_path}/conversation/{conversation_2_id}/message', json=
{
  "parts": [
    {
      "type": "TEXT",
      "content": "What is the secret of Mr. DataMcDaFace?"
    }
  ],
  "overrides": []
})

stream_response(r)